In [2]:
#IMPORTA LAS LIBRERIAS A UTILIZAR
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [0]:
#CARGA DOCUMENTOS DE ARCHIVOS .CSV
df_holidays = pd.read_csv("holidays_events.csv")
df_oil = pd.read_csv("oil.csv")
df_stores = pd.read_csv("stores.csv")
df_transactions = pd.read_csv("transactions.csv")
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [0]:
# TRANSFORM

#OBSERVAR QUE LA COLUMNA FECHA NO ESTA EN FORMATO FECHA
df_holidays.dtypes
df_oil.dtypes
df_stores.dtypes
df_oil.dtypes
df_transactions.dtypes
df_train.dtypes
df_test.dtypes

In [0]:
#CONVERTIR EL STRING EN FORMATO FECHA PARA CADA ARCHIVO

df_holidays['date'] = pd.to_datetime(df_holidays['date'])
df_oil['date'] = pd.to_datetime(df_oil['date'])
#df_storeS no tiene variable
df_transactions['date'] = pd.to_datetime(df_transactions['date'])
df_train['date'] = pd.to_datetime(df_train['date'])
df_test['date'] = pd.to_datetime(df_test['date'])

In [0]:
### OBSERVAR LOS CAMBIOS

df_holidays.info()
print("\n")
df_oil.info()
print("\n")
#df_stores no tiene variable fecha
df_transactions.info()
print("\n")
df_train.info()
print("\n")
df_test.info()

In [0]:
#DROP DE INFORMACION INNECESARIA
df_holidays.drop(["locale", "locale_name", "description", "transferred"], axis = 1, inplace = True)
df_holidays.info()

In [0]:
#DROP DE INFORMACION INNECESARIA
df_stores.drop(["type", "cluster"], axis = 1, inplace = True)
df_stores.info()

In [0]:
#DROP DE INFORMACION INNECESARIA
df_test.drop(["id", "family"], axis = 1, inplace = True)
#DROP DE INFORMACION INNECESARIA
df_train.drop(["id", "family"], axis = 1, inplace = True)
df_train.head()


In [0]:
# IMPRIME DATOS DE DOCUMENTO HOLIDAYS
df_holidays = pd.read_csv('holidays_events.csv')
df_holidays.head()

In [0]:
unique_local = df_holidays['locale_name'].unique()
local_count = df_holidays['locale_name'].value_counts()

plt.bar(unique_local, local_count)
plt.xlabel('Localities')
plt.ylabel('N° of Holidays')
plt.title('Holidays per Locality')
plt.xticks(rotation='vertical')  # Rotate x-axis labels vertically
plt.tight_layout()
plt.show()

In [0]:
#PLOT DONDE SE MUESTA GRAFICADA LA TOTALIDA DE LA INFORMACION EN BUSQUEDA DE PATRONES


df_transactions = df_transactions[['date', 'transactions']]
df_types = df_holidays[['date', 'type']]

# CONVERTIR LAS COLUMNAS 'FECHA' A OBJETOS DATETIME
df_transactions['date'] = pd.to_datetime(df_transactions['date'], format='%Y-%m-%d')
df_types['date'] = pd.to_datetime(df_types['date'], format='%Y-%m-%d')

#COMBINAR DATAFRAMES
merged_df = df_transactions.merge(df_types, on='date', how='left')

# CALCULA LA MEDIA DE TRANSACTIONS
mean_transactions = merged_df['transactions'].mean()

# USA SEABORN PARA GRAFICAR LOS VALORES PREVIOS
plt.figure(figsize=(10, 6))
sns.lineplot(data=merged_df, x='date', y='transactions', hue='type')
plt.axhline(mean_transactions, color='red', linestyle='dashed', label='Mean Transactions')
plt.title('Transactions over Time')
plt.xlabel('Date')
plt.ylabel('Transactions')
plt.legend(title='Type')
'''
# Limit x-axis to a specific time frame
start_date = '2017-01-01'
end_date = '2018-01-01'
plt.xlim(pd.to_datetime(start_date), pd.to_datetime(end_date))
'''
plt.show()

In [0]:
# GRAFICA DE DATOS DEL AÑO 2016 a 2017 DONDE SE COMPARA EL AÑO DEL TERREMOTO Y EL POSTERIOR
plt.figure(figsize=(10, 6))
sns.lineplot(data=merged_df, x='date', y='transactions', hue='type')
plt.axhline(mean_transactions, color='red', linestyle='dashed', label='Mean Transactions')
plt.title('Transactions over Time')
plt.xlabel('Date')
plt.ylabel('Transactions')
plt.legend(title='Type')

# LIMITA LOS EJES PARA ENFATIZAR LOS AÑOS 2016 y 2017
start_date = '2016-01-01'
end_date = '2017-01-01'
plt.xlim(pd.to_datetime(start_date), pd.to_datetime(end_date))

In [0]:
# GRAFICA DE DATOS POSTERIORES A EVENTOS DE GRAN IMPACTO TERREMOTO(2016)
plt.figure(figsize=(10, 6))
sns.lineplot(data=merged_df, x='date', y='transactions', hue='type')

plt.axhline(mean_transactions, color='red', linestyle='dashed', label='Mean Transactions')
plt.title('Transactions over Time')
plt.xlabel('Date')
plt.ylabel('Transactions')
plt.legend(title='Type')

# LIMITA LOS EJES PARA MOSTRAR SOLO DESDE 2017 y 2018
start_date = '2017-01-01'
end_date = '2018-01-01'
plt.xlim(pd.to_datetime(start_date), pd.to_datetime(end_date))
plt.show()

In [0]:
df_transactions = df_transactions[['date', 'transactions']]
df_types = df_oil[['date', 'dcoilwtico']]

# Convert 'date' columns to datetime objects
df_transactions['date'] = pd.to_datetime(df_transactions['date'], format='%Y-%m-%d')
df_types['date'] = pd.to_datetime(df_types['date'], format='%Y-%m-%d')

# Plot using seaborn
plt.plot(df_transactions['date'], df_transactions['transactions'], label='Transactions per day')
plt.plot(df_types['date'], df_types['dcoilwtico'], label='Oil price')
plt.xlabel('Date')
plt.ylabel('')
plt.title('Oil price and Trasactions')
plt.legend()
plt.show()

In [0]:
# STORES DATAFRAME

df_stores = pd.read_csv("stores.csv")
df_stores.head()
df_stores.info()
#df_stores.isnull()
#df_stores.count()
new_df_stores = df_stores.drop('type', axis = 'columns')